# __8.1 Considering topical impact__

Goal:
- Access the impacts of pubs over time (1999-2020) for different:
  - Topics
  - Countries in 8.2

Approach
- For each country/sp/toc for each year, determine:
  - Average impact score
- Before calculate average, impact scores are processed for each year
  - Rank
    - Inverted, min-max normalized, then squared
    - Squared to create more distance between top ranks from bottome ranks
  - The rest not normalized because there can be significant differences in the max values across years which lead to anomalous results:
    - SJR
    - H index
    - Cites / Doc. (2years)
    - For anomalous results, see `impact_topics_MOD_ANOMALY.xlsx`

Data source
- [Medline journal list](https://ftp.ncbi.nih.gov/pubmed/J_Medline.txt)
- [Scimago Journal & Country Rank](https://www.scimagojr.com/) for journal ranking data

Key number:
- Pubmed journal
  - \# with jtitle and ISSN: 31624
  - \# with jtitle, no ISSN: 4257
  - \# with ISSN, no jtitle: 0
  - 23 ISSNs are assigned to >=2 JournalTitle
  - Some JournalTitles are assigned to 2 ISSNs beyond print/online.
- Of 421307 records, 163's journal names are not consistent with those in Medline.

## ___Setup___

### Module import

In conda env `base`

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
from urllib import request
from time import sleep

### Key variables

In [2]:
# Reproducibility
seed = 20220609

# Setting working directory
#proj_dir   = Path.home() / "projects/plant_sci_hist"
#work_dir   = proj_dir / "8_impact/8_1_topic"
work_dir = Path('/mnt/home/ande2472/github/pubmed_topics/test_files')
work_dir.mkdir(parents=True, exist_ok=True)

# # plant science corpus with topic assignment info
# dir42      = proj_dir / "4_topic_model/4_2_outlier_assign"
# corpus_file = dir42 / "table4_2_corpus_with_topic_assignment.tsv.gz"
# #corpus_file = dir42 / "test.tsv"

# # timestamp bins
# dir44            = proj_dir / "4_topic_model/4_4_over_time"
# ts_for_bins_file = dir44 / "table4_4_bin_timestamp_date.tsv"
# file_topic_name   = dir44 / "fig4_4_tot_heatmap_weighted_xscaled_names.txt"

# # country data
# dir74            = proj_dir / "7_countries/7_4_consolidate_all"
# ci_file          = dir74 / 'country_info_final_a3.txt'



# # So PDF is saved in a format properly
# mpl.rcParams['pdf.fonttype'] = 42
# plt.rcParams["font.family"] = "sans-serif"

## ___Journal list___

### Download

Example
```
--------------------------------------------------------
JrId: 1
JournalTitle: AADE editors' journal
jtitle: AADE Ed J
ISSN (Print): 0160-6999
ISSN (Online): 
IsoAbbr: AADE Ed J
NlmId: 7708172
--------------------------------------------------------
JrId: 2
...
```

In [5]:
# Get journal list
jm_url  = "https://ftp.ncbi.nih.gov/pubmed/J_Medline.txt"
jm_file = work_dir / "J_Medline.txt"
if not jm_file.exists():
  _ = request.urlretrieve(jm_url, jm_file)

In [7]:
jm_file

PosixPath('/mnt/home/ande2472/github/pubmed_topics/test_files/J_Medline.txt')

### Establish journal dictionary

Considerations
- The ISSN number has the formmat `xxxx-xxxx`, rid of `-` to simplfy matching with SJR data.
- Some journal names have multiple ISSNs, and some ISSNs refer to different journal names.

In [4]:
def add_to_dict(i_to_j, j_to_i, issn, jtitle):
  if issn != "":
    if issn not in i_to_j:
      i_to_j[issn] = [jtitle]
    else:
      redun_issn.append([issn, jtitle, i_to_j[issn]])
      i_to_j[issn].append(jtitle)

    if jtitle not in j_to_i:
      j_to_i[jtitle] = [issn]
    else:
      j_to_i[jtitle].append(issn)

  return i_to_j, j_to_i

In [5]:
with open(jm_file) as f:
  i_to_j = {} # {ISSN: [JournalTitle]}
  j_to_i = {} # {JournalTitle: [ISSN]}

  # issn : issn print
  # issn2: issn online
  jtitle = issn = issn2 = ""
  wi_m_no_i = wi_i_no_m = 0
  redun_issn   = []
  f.readline() # rid of 1st line
  for line in f:
    # new record
    if line.startswith("----"):
      if jtitle != "" and (issn != "" or issn2 != ""):
        i_to_j, j_to_i = add_to_dict(i_to_j, j_to_i, issn, jtitle)
        if issn != issn2:
          i_to_j, j_to_i = add_to_dict(i_to_j, j_to_i, issn2, jtitle)

      if jtitle == "":
        wi_i_no_m += 1
        print("With ISSN, no jtitle:", issn, issn2)
      if issn == "" and issn2 == "":
        wi_m_no_i += 1
      # reset
      jtitle = issn = issn2 = ""
    elif line.startswith("JournalTitle"):
      jtitle_tokens = line.strip().split("JournalTitle: ")
      if len(jtitle_tokens) == 2:
        jtitle = jtitle_tokens[1]
    elif line.startswith("ISSN (Print)"):
      issn_tokens = line.strip().split("ISSN (Print): ")
      if len(issn_tokens) == 2:
        issn = "".join(issn_tokens[1].split("-"))
    elif line.startswith("ISSN (Online)"):
      issn2_tokens = line.strip().split("ISSN (Online): ")
      if len(issn2_tokens) == 2:
        issn2 = "".join(issn2_tokens[1].split("-"))

# Add the last record
if jtitle != "" and (issn != "" or issn2 != ""):
  i_to_j, j_to_i = add_to_dict(i_to_j, j_to_i, issn, jtitle)
  if issn != issn2:
    i_to_j, j_to_i = add_to_dict(i_to_j, j_to_i, issn2, jtitle)

if jtitle == "":
  wi_i_no_m += 1
  print("With ISSN, no jtitle:", issn, issn2)
if issn == "" and issn2 == "":
  wi_m_no_i += 1

In [12]:
len(redun_issn), len(i_to_j), len(j_to_i), wi_m_no_i, wi_i_no_m

(22, 44699, 31939, 4280, 0)

In [55]:
wi_m_no_i

4280

In [15]:
i_to_j['00664154']

['Annual review of biochemistry']

In [54]:
redun_issn

[['07306652',
  'Journal of cellular biochemistry. Supplement',
  ['Journal of supramolecular structure and cellular biochemistry. Supplement',
   'Journal of cellular biochemistry. Supplement']],
 ['07077270',
  'The Journal of otolaryngology. Supplement',
  ['The Journal of otolaryngology',
   'The Journal of otolaryngology. Supplement']],
 ['02785846',
  'Progress in neuro-psychopharmacology',
  ['Progress in neuro-psychopharmacology & biological psychiatry',
   'Progress in neuro-psychopharmacology']],
 ['10969888',
  'Journal of mass spectrometry : JMS',
  ['Biological mass spectrometry', 'Journal of mass spectrometry : JMS']],
 ['03027430',
  'JBR-BTR : organe de la Societe royale belge de radiologie (SRBR) = orgaan van de Koninklijke Belgische Vereniging voor Radiologie (KBVR)',
  ['Journal belge de radiologie',
   'JBR-BTR : organe de la Societe royale belge de radiologie (SRBR) = orgaan van de Koninklijke Belgische Vereniging voor Radiologie (KBVR)']],
 ['03855716',
  'Memai h

## ___Journal impact data___ 

### Download

__IMPORTANT__: SJR data use "," for decimal point!!!!


In [9]:
sjr_base_url = "https://www.scimagojr.com/journalrank.php?out=xls&year="

sjr_dir   = work_dir / "_sjr"
sjr_dir.mkdir(parents=True, exist_ok=True)
yr_range  = range(1999,2021)
for year in tqdm(yr_range):
  sjr_file = sjr_dir / f"scimagojr_{year}.csv"
  if not sjr_file.exists():
    _ = request.urlretrieve(sjr_base_url + str(year), sjr_file)
    sleep(5)


100%|██████████| 22/22 [00:00<00:00, 1450.20it/s]


### Check out sjr data

In [10]:
sjr_file = sjr_dir / "scimagojr_2020.csv"
sjr2020 = pd.read_csv(sjr_file, sep=";")
sjr2020

/tmp/ipykernel_250712/257926905.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  sjr2020 = pd.read_csv(sjr_file, sep=";")


,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2020),Total Docs. (3years),...,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Areas
0,1,28773,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","62,937",Q1,198,45,118,...,15496,79,"127,78","73,53",United States,Northern America,Wiley-Blackwell,1950-2022,Hematology (Q1); Oncology (Q1),Medicine
1,2,19434,MMWR Recommendations and Reports,journal,"10575987, 15458601","40,949",Q1,151,10,9,...,501,9,"53,00","129,20",United States,Northern America,Centers for Disease Control and Prevention (CDC),1990-2022,Epidemiology (Q1); Health Information Manageme...,Environmental Science; Health Professions; Med...
2,3,20315,Nature Reviews Molecular Cell Biology,journal,"14710072, 14710080","37,461",Q1,485,114,338,...,10530,166,"31,99","73,21",United Kingdom,Western Europe,Nature Publishing Group,2000-2022,Cell Biology (Q1); Molecular Biology (Q1),"Biochemistry, Genetics and Molecular Biology"
3,4,29431,Quarterly Journal of Economics,journal,"00335533, 15314650","34,573",Q1,292,40,113,...,1818,112,"14,54","68,33",United Kingdom,Western Europe,Oxford University Press,1886-2022,Economics and Econometrics (Q1),"Economics, Econometrics and Finance"
4,5,21100812243,Nature Reviews Materials,journal,20588437,"32,011",Q1,156,86,263,...,10557,137,"30,16","109,55",United Kingdom,Western Europe,Nature Publishing Group,2016-2022,"Biomaterials (Q1); Electronic, Optical and Mag...",Energy; Materials Science
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35021,35022,19324,Zhonghua wai ke za zhi [Chinese journal of sur...,journal,05295815,NaN,-,18,5,0,...,0,0,"0,00","0,00",China,Asiatic Region,Zhonghua Yixuehui Zazhishe,"1957, 1959-1964, 1979-2016, 2020",Medicine (miscellaneous),Medicine
35022,35023,21101021861,Zhurnal Belorusskogo Gosudarstvennogo Universi...,journal,"25206508, 26173956",NaN,-,4,30,88,...,27,88,"0,36","11,57",Belarus,Eastern Europe,The Belarusian State University,2017-2022,Algebra and Number Theory; Computational Theor...,Computer Science; Mathematics
35023,35024,20906,Zhurnal Mikrobiologii Epidemiologii i Immunobi...,journal,"26867613, 03729311",NaN,-,12,64,97,...,25,97,"0,26","26,03",Russian Federation,Eastern Europe,Central Research Institute for Epidemiology,"1945-1947, 1954-2016, 2020-2022",Immunology; Medicine (miscellaneous); Microbio...,Immunology and Microbiology; Medicine
35024,35025,21101023406,Zoodiversity,journal,"2707725X, 27077268",NaN,-,15,50,159,...,99,159,"0,61","33,60",Ukraine,Eastern Europe,National Academy of Sciences of Ukraine,2020-2022,"Animal Science and Zoology; Ecology, Evolution...",Agricultural and Biological Sciences


In [116]:
type(sjr2020['Issn'][4])

str

In [57]:
df.columns

Index(['Rank', 'Sourceid', 'Title', 'Type', 'Issn', 'SJR', 'SJR Best Quartile',
       'H index', 'Total Docs. (2001)', 'Total Docs. (3years)', 'Total Refs.',
       'Total Cites (3years)', 'Citable Docs. (3years)',
       'Cites / Doc. (2years)', 'Ref. / Doc.', 'Country', 'Region',
       'Publisher', 'Coverage', 'Categories', 'Areas'],
      dtype='object')

In [3]:
journals = pd.read_csv("/mnt/home/ande2472/data/full_clean_data/journals.csv", sep='\t')


In [4]:
journals

,Journal,Article Count,Cumulative Count,Total Percentage,Cumulative Percentage
0,PloS one,265526,265526,1.130048e-02,0.011300
1,Scientific reports,164163,429689,6.986590e-03,0.018287
2,The Journal of biological chemistry,155026,584715,6.597730e-03,0.024885
3,Proceedings of the National Academy of Science...,129027,713742,5.491242e-03,0.030376
4,Biochemical and biophysical research communica...,71993,785735,3.063940e-03,0.033440
...,...,...,...,...,...
24166,Journal of neuropathic pain &amp; symptom pall...,1,23496867,4.255886e-08,1.000000
24167,"Journal of neurology, neurosurgery and spine",1,23496868,4.255886e-08,1.000000
24168,"Journal of neurology research, reviews &amp; r...",1,23496869,4.255886e-08,1.000000
24169,Journal of neurology and neurorehabilitation r...,1,23496870,4.255886e-08,1.000000


In [8]:
def matchISSN(journals, j_to_i):
    newjournals = journals
    newjournals["ISSN"] = 999
    for i in range(len(newjournals)):
        if "&amp;" in newjournals["Journal"][i]:
           # problem = journals["Journal"][i]
           # solution = journals["Journal"][i].replace('&amp;', '&')
           # print(f"Changing {problem} to {solution}")
            newjournals["Journal"][i] = newjournals["Journal"][i].replace('&amp;', '&')
        try:
            issnlist = j_to_i[newjournals["Journal"][i]]
            newjournals["ISSN"][i] = issnlist
            # if len(issnlist) == 2:
            #     newjournals["ISSN"][i] = issnlist[1] + ','+ issnlist[0]
            # elif len(issnlist) == 1:
            #     newjournals["ISSN"][i] = f'{issnlist[0]}'
            # else:
            #     problem = newjournals["Journal"][i]
            #     print(f"Irregularity on: {issnlist} with journal {problem}")
            #     newjournals["ISSN"][i] = 666
        except:
            problem = newjournals["Journal"][i]
            print(f"No match found for {problem}. Skipping")
            
    return newjournals



In [11]:
def addSRJ(journals, years, sjr_dir):
    sjrjournals = journals
    sjrjournals = sjrjournals.assign(issn=sjrjournals['ISSN']).explode('issn')
    for year in years:
        sjr_file = sjr_dir / f"scimagojr_{year}.csv"
        sjr = pd.read_csv(sjr_file, sep=";")
        yearname = f'sjr{year}'
        sjr['ParsedISSN'] = 999
        for i in range(len(sjr)):
            sjr['ParsedISSN'][i] = sjr['Issn'][i].split(', ')
        sjr_exploded = sjr.assign(issn=sjr['ParsedISSN']).explode('issn')
        sjrjournals[yearname] = -1
        
        for i in range(len(sjrjournals)):
            for issn in sjrjournals['issn'][i]:
                print(issn)
                try:
                    sjrjournals[yearname][i] = sjr_exploded[sjr_exploded['issn']==issn]['SJR'][0]
                except:
                    print(f'yikes, no matches found for ISSN: {issn}')

    return sjrjournals
        
        # newjournals[yearname] = 0
        # for i in range(len(newjournals)):
        #     try:
        #         newjournals[yearname]
        
    

In [ ]:
journalsISSN = matchISSN(journals, j_to_i)

In [15]:
sum(journalsISSN["ISSN"]==999)

1539

In [113]:
years = [2020]
sjrjournals = journalsISSN
sjrjournals = sjrjournals.assign(issn=sjrjournals['ISSN']).explode('issn')
sjrjournals = sjrjournals.reset_index()
for year in years:
    sjr_file = sjr_dir / f"scimagojr_{year}.csv"
    sjr = pd.read_csv(sjr_file, sep=";")
    yearname = f'sjr{year}'
    sjr['ParsedISSN'] = 999
    for i in range(len(sjr)):
        sjr['ParsedISSN'][i] = sjr['Issn'][i].split(', ')
    sjr_exploded = sjr.assign(issn=sjr['ParsedISSN']).explode('issn')
    sjr_exploded = sjr_exploded.reset_index()
    sjrjournals[yearname] = -1
    
    for i in range(len(sjrjournals)):
        issn = sjrjournals['issn'][i]
        if issn != 999:
            try:
                sjrjournals[yearname][i] = int(sjr_exploded[sjr_exploded['issn']==issn]['SJR'].values[0].replace(',', ''))
            except:
                print(f'yikes, no matches found for ISSN: {issn}')
        else:
            sjrjournals[yearname][i] = 1


/tmp/ipykernel_269976/3090504383.py:7: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  sjr = pd.read_csv(sjr_file, sep=";")
/tmp/ipykernel_269976/3090504383.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sjr['ParsedISSN'][i] = sjr['Issn'][i].split(', ')
/tmp/ipykernel_269976/3090504383.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sjrjournals[yearname][i] = int(sjr_exploded[sjr_exploded['issn']==issn]['SJR'].values[0].replace(',', ''))


yikes, no matches found for ISSN: 00063002
yikes, no matches found for ISSN: 18782434
yikes, no matches found for ISSN: 15394522
yikes, no matches found for ISSN: 00029513
yikes, no matches found for ISSN: 21635773
yikes, no matches found for ISSN: 15393755
yikes, no matches found for ISSN: 15502376
yikes, no matches found for ISSN: 18791298
yikes, no matches found for ISSN: 21688184
yikes, no matches found for ISSN: 23757477
yikes, no matches found for ISSN: 26940604
yikes, no matches found for ISSN: 18733336
yikes, no matches found for ISSN: 16005368
yikes, no matches found for ISSN: 00142956
yikes, no matches found for ISSN: 22192840
yikes, no matches found for ISSN: 22148019
yikes, no matches found for ISSN: 20770383
yikes, no matches found for ISSN: 15334880
yikes, no matches found for ISSN: 15334899
yikes, no matches found for ISSN: 25144138
yikes, no matches found for ISSN: 00296570
yikes, no matches found for ISSN: 20479018
yikes, no matches found for ISSN: 18733557
yikes, no m

/tmp/ipykernel_269976/3090504383.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sjrjournals[yearname][i] = 1


yikes, no matches found for ISSN: 10968628
yikes, no matches found for ISSN: 18732763
yikes, no matches found for ISSN: 15590291
yikes, no matches found for ISSN: 15564029
yikes, no matches found for ISSN: 15440591
yikes, no matches found for ISSN: 09598138
yikes, no matches found for ISSN: 18787568
yikes, no matches found for ISSN: 03729311
yikes, no matches found for ISSN: 26867613
yikes, no matches found for ISSN: 18727786
yikes, no matches found for ISSN: 00039950
yikes, no matches found for ISSN: 15383601
yikes, no matches found for ISSN: 17424658
yikes, no matches found for ISSN: 00268925
yikes, no matches found for ISSN: 16386183
yikes, no matches found for ISSN: 00040010
yikes, no matches found for ISSN: 15383644
yikes, no matches found for ISSN: 2249782X
yikes, no matches found for ISSN: 0973709X
yikes, no matches found for ISSN: 00367672
yikes, no matches found for ISSN: 14735644
yikes, no matches found for ISSN: 05295815
yikes, no matches found for ISSN: 15524965
yikes, no m

990

,index,Journal,Article Count,Cumulative Count,Total Percentage,Cumulative Percentage,ISSN,issn,sjr2020
0,0,PloS one,265526,265526,1.130048e-02,0.011300,[19326203],19326203,990
1,1,Scientific reports,164163,429689,6.986590e-03,0.018287,[20452322],20452322,1240
2,2,The Journal of biological chemistry,155026,584715,6.597730e-03,0.024885,"[00219258, 1083351X]",00219258,2361
3,2,The Journal of biological chemistry,155026,584715,6.597730e-03,0.024885,"[00219258, 1083351X]",1083351X,2361
4,3,Proceedings of the National Academy of Science...,129027,713742,5.491242e-03,0.030376,"[00278424, 10916490]",00278424,5011
...,...,...,...,...,...,...,...,...,...
36178,24166,Journal of neuropathic pain & symptom palliation,1,23496867,4.255886e-08,1.000000,"[15437698, 15437701]",15437701,-1
36179,24167,"Journal of neurology, neurosurgery and spine",1,23496868,4.255886e-08,1.000000,999,999,-1
36180,24168,"Journal of neurology research, reviews & reports",1,23496869,4.255886e-08,1.000000,999,999,-1
36181,24169,Journal of neurology and neurorehabilitation r...,1,23496870,4.255886e-08,1.000000,999,999,-1


In [129]:
journalranks = sjrjournals.drop(['issn', 'index'], axis=1).groupby('Journal', as_index=False).agg({'sjr2020': 'max', 'Article Count': 'mean'})

In [142]:

journalranks["Article Count"] = journalranks["Article Count"].astype(int)
sortedranks = journalranks.sort_values('sjr2020', ascending=False).reset_index()
sortedranks['Cumulative Count'] = sortedranks['Article Count'].cumsum()
sortedranks['Total Percentage'] = sortedranks['Article Count']/sortedranks['Article Count'].sum()
sortedranks['Cumulative Percentage'] = sortedranks['Total Percentage'].cumsum()

In [152]:
#Top 250 = 979189
#Top 400 = 1461288
#Top 500 = 1895763
sortedranks['Cumulative Count'][400]

1461288

In [155]:
sortedranks.to_csv('/mnt/home/ande2472/data/full_clean_data/sortedranks.csv', sep='\t')

In [118]:
sum(journalranks['sjr2020'] == 1)

1539

In [112]:
sum(journalranks[journalranks['issn']==999])

KeyError: 'issn'

In [93]:
import re

In [104]:
string = 'IEEE conference on high performance extreme computing. IEEE Conference on High Performance Extreme Computing'

In [105]:
res = [val for key, val in j_to_i.items() if re.search(string, key)]

In [106]:
res

[]

In [107]:
j_to_i['IEEE conference on high performance extreme computing. IEEE Conference on High Performance Extreme Computing']

KeyError: 'IEEE conference on high performance extreme computing. IEEE Conference on High Performance Extreme Computing'

In [75]:
sjrjournals.groupby('Journal').agg({'sjr2020': 'max'})

Journal
... Asian Conference on Pattern Recognition. Asian Conference on Pattern Recognition                                                -1
... Electronic Components & Technology Conference. Electronic Components & Technology Conference                                    -1
... Health innovations and point-of-care technologies conference. Health innovations and point-of-care technologies conference      -1
... IEEE Conference on Collaboration and Internet Computing. IEEE Conference on Collaboration and Internet Computing                -1
... IEEE Conference on Communications and Network Security. IEEE Conference on Communications and Network Security                  -1
                                                                                                                                  ... 
microPublication biology                                                                                                            -1
microRNA diagnostics and therapeutics          

In [76]:
sjrjournals

,index,Journal,Article Count,Cumulative Count,Total Percentage,Cumulative Percentage,ISSN,issn,sjr2020
0,0,PloS one,265526,265526,1.130048e-02,0.011300,[19326203],19326203,990
1,1,Scientific reports,164163,429689,6.986590e-03,0.018287,[20452322],20452322,1240
2,2,The Journal of biological chemistry,155026,584715,6.597730e-03,0.024885,"[00219258, 1083351X]",00219258,2361
3,2,The Journal of biological chemistry,155026,584715,6.597730e-03,0.024885,"[00219258, 1083351X]",1083351X,2361
4,3,Proceedings of the National Academy of Science...,129027,713742,5.491242e-03,0.030376,"[00278424, 10916490]",00278424,5011
...,...,...,...,...,...,...,...,...,...
36178,24166,Journal of neuropathic pain & symptom palliation,1,23496867,4.255886e-08,1.000000,"[15437698, 15437701]",15437701,-1
36179,24167,"Journal of neurology, neurosurgery and spine",1,23496868,4.255886e-08,1.000000,999,999,-1
36180,24168,"Journal of neurology research, reviews & reports",1,23496869,4.255886e-08,1.000000,999,999,-1
36181,24169,Journal of neurology and neurorehabilitation r...,1,23496870,4.255886e-08,1.000000,999,999,-1


In [61]:
i_to_j['1557170X']

['Conference proceedings : ... Annual International Conference of the IEEE Engineering in Medicine and Biology Society. IEEE Engineering in Medicine and Biology Society. Annual Conference']

In [59]:
j_to_i['Annual International Conference of the IEEE Engineering in Medicine and Biology Society. IEEE Engineering in Medicine and Biology Society. Annual International Conference']

['23757477', '26940604']

In [42]:
sjr_exploded[sjr_exploded['issn']==issn]['SJR'].values[0]

'2,852'

In [110]:
sum(sjrjournals['sjr2020'] == -1)

14402

In [10]:
journalsSJR = addSRJ(journals, [2020], sjr_dir)

NameError: name 'addSRJ' is not defined

NameError: name 'journalsSJR' is not defined

In [150]:
df_exploded[df_exploded['issn']=='15424863']['SJR'][0]

'62,937'

In [134]:
df_exploded = newSJR.assign(issn=newSJR['ParsedISSN']).explode('issn')


In [143]:
df_exploded['issn'][df_exploded['issn']=='15424863'][0]

'15424863'

In [133]:
x = '15424863'
newSJR[x in newSJR['ParsedISSN']]

KeyError: False

['15424863', ' 00079235']

In [130]:
newSJR = addSRJ(journals, [2020], sjr_dir)

/tmp/ipykernel_90622/1124162737.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  sjr = pd.read_csv(sjr_file, sep=";")
/tmp/ipykernel_90622/1124162737.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sjr['ParsedISSN'][i] = sjr['Issn'][i].split(', ')


In [131]:
newSJR

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2020),Total Docs. (3years),...,Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Areas,ParsedISSN
0,1,28773,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","62,937",Q1,198,45,118,...,79,"127,78","73,53",United States,Northern America,Wiley-Blackwell,1950-2022,Hematology (Q1); Oncology (Q1),Medicine,"[15424863, 00079235]"
1,2,19434,MMWR Recommendations and Reports,journal,"10575987, 15458601","40,949",Q1,151,10,9,...,9,"53,00","129,20",United States,Northern America,Centers for Disease Control and Prevention (CDC),1990-2022,Epidemiology (Q1); Health Information Manageme...,Environmental Science; Health Professions; Med...,"[10575987, 15458601]"
2,3,20315,Nature Reviews Molecular Cell Biology,journal,"14710072, 14710080","37,461",Q1,485,114,338,...,166,"31,99","73,21",United Kingdom,Western Europe,Nature Publishing Group,2000-2022,Cell Biology (Q1); Molecular Biology (Q1),"Biochemistry, Genetics and Molecular Biology","[14710072, 14710080]"
3,4,29431,Quarterly Journal of Economics,journal,"00335533, 15314650","34,573",Q1,292,40,113,...,112,"14,54","68,33",United Kingdom,Western Europe,Oxford University Press,1886-2022,Economics and Econometrics (Q1),"Economics, Econometrics and Finance","[00335533, 15314650]"
4,5,21100812243,Nature Reviews Materials,journal,20588437,"32,011",Q1,156,86,263,...,137,"30,16","109,55",United Kingdom,Western Europe,Nature Publishing Group,2016-2022,"Biomaterials (Q1); Electronic, Optical and Mag...",Energy; Materials Science,[20588437]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35021,35022,19324,Zhonghua wai ke za zhi [Chinese journal of sur...,journal,05295815,NaN,-,18,5,0,...,0,"0,00","0,00",China,Asiatic Region,Zhonghua Yixuehui Zazhishe,"1957, 1959-1964, 1979-2016, 2020",Medicine (miscellaneous),Medicine,[05295815]
35022,35023,21101021861,Zhurnal Belorusskogo Gosudarstvennogo Universi...,journal,"25206508, 26173956",NaN,-,4,30,88,...,88,"0,36","11,57",Belarus,Eastern Europe,The Belarusian State University,2017-2022,Algebra and Number Theory; Computational Theor...,Computer Science; Mathematics,"[25206508, 26173956]"
35023,35024,20906,Zhurnal Mikrobiologii Epidemiologii i Immunobi...,journal,"26867613, 03729311",NaN,-,12,64,97,...,97,"0,26","26,03",Russian Federation,Eastern Europe,Central Research Institute for Epidemiology,"1945-1947, 1954-2016, 2020-2022",Immunology; Medicine (miscellaneous); Microbio...,Immunology and Microbiology; Medicine,"[26867613, 03729311]"
35024,35025,21101023406,Zoodiversity,journal,"2707725X, 27077268",NaN,-,15,50,159,...,159,"0,61","33,60",Ukraine,Eastern Europe,National Academy of Sciences of Ukraine,2020-2022,"Animal Science and Zoology; Ecology, Evolution...",Agricultural and Biological Sciences,"[2707725X, 27077268]"


In [89]:
newjournals = matchISSN(journals,j_to_i)

/tmp/ipykernel_90622/1735710365.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newjournals["ISSN"][i] = issnlist


No match found for Journal of chromatography. Skipping
No match found for Revista espanola de enfermedades digestivas : organo oficial de la Sociedad Espanola de Patologia Digestiva. Skipping
No match found for Neurogastroenterology and motility : the official journal of the European Gastrointestinal Motility Society. Skipping
No match found for Medical image computing and computer-assisted intervention : MICCAI ... International Conference on Medical Image Computing and Computer-Assisted Intervention. Skipping
No match found for International journal of obesity and related metabolic disorders : journal of the International Association for the Study of Obesity. Skipping
No match found for La semaine des hopitaux : organe fonde par l'Association d'enseignement medical des hopitaux de Paris. Skipping
No match found for Polskie Archiwum Medycyny Wewnetrznej. Skipping
No match found for Ai zheng = Aizheng = Chinese journal of cancer. Skipping
No match found for Lin chuang er bi yan hou ke 

In [90]:
newjournals

,Journal,Article Count,Cumulative Count,Total Percentage,Cumulative Percentage,ISSN
0,PloS one,265526,265526,1.130048e-02,0.011300,[19326203]
1,Scientific reports,164163,429689,6.986590e-03,0.018287,[20452322]
2,The Journal of biological chemistry,155026,584715,6.597730e-03,0.024885,"[00219258, 1083351X]"
3,Proceedings of the National Academy of Science...,129027,713742,5.491242e-03,0.030376,"[00278424, 10916490]"
4,Biochemical and biophysical research communica...,71993,785735,3.063940e-03,0.033440,"[0006291X, 10902104]"
...,...,...,...,...,...,...
24166,Journal of neuropathic pain & symptom palliation,1,23496867,4.255886e-08,1.000000,"[15437698, 15437701]"
24167,"Journal of neurology, neurosurgery and spine",1,23496868,4.255886e-08,1.000000,999
24168,"Journal of neurology research, reviews & reports",1,23496869,4.255886e-08,1.000000,999
24169,Journal of neurology and neurorehabilitation r...,1,23496870,4.255886e-08,1.000000,999


In [92]:
newjournals[newjournals["ISSN"] == 999].head(20)

,Journal,Article Count,Cumulative Count,Total Percentage,Cumulative Percentage,ISSN
472,Journal of chromatography,8831,8868839,0.000376,0.377448,999
1358,Revista espanola de enfermedades digestivas : ...,4167,14084967,0.000177,0.599440,999
1525,Neurogastroenterology and motility : the offic...,3782,14747163,0.000161,0.627622,999
2040,Medical image computing and computer-assisted ...,2874,16443517,0.000122,0.699817,999
2219,International journal of obesity and related m...,2659,16936305,0.000113,0.720790,999
2363,La semaine des hopitaux : organe fonde par l'A...,2495,17307224,0.000106,0.736576,999
2458,Polskie Archiwum Medycyny Wewnetrznej,2404,17539728,0.000102,0.746471,999
2556,Ai zheng = Aizheng = Chinese journal of cancer,2312,17770692,0.000098,0.756300,999
3248,Lin chuang er bi yan hou ke za zhi = Journal o...,1759,19167324,0.000075,0.815739,999
3265,Medicinski arhiv,1748,19197126,0.000074,0.817008,999


In [77]:
list1 = ['00664154', '15454509']
list2 = ['15454509', '00664154']
list1 == list2

False

In [72]:
newjournals

,Journal,Article Count,Cumulative Count,Total Percentage,Cumulative Percentage,ISSN
0,PloS one,265526,265526,1.130048e-02,0.011300,[19326203]
1,Scientific reports,164163,429689,6.986590e-03,0.018287,[20452322]
2,The Journal of biological chemistry,155026,584715,6.597730e-03,0.024885,"[00219258, 1083351X]"
3,Proceedings of the National Academy of Science...,129027,713742,5.491242e-03,0.030376,"[00278424, 10916490]"
4,Biochemical and biophysical research communica...,71993,785735,3.063940e-03,0.033440,"[0006291X, 10902104]"
...,...,...,...,...,...,...
24166,Journal of neuropathic pain & symptom palliation,1,23496867,4.255886e-08,1.000000,"[15437698, 15437701]"
24167,"Journal of neurology, neurosurgery and spine",1,23496868,4.255886e-08,1.000000,999
24168,"Journal of neurology research, reviews & reports",1,23496869,4.255886e-08,1.000000,999
24169,Journal of neurology and neurorehabilitation r...,1,23496870,4.255886e-08,1.000000,999


### Functions for processing SJR data

Orignally, all metric in the SJR data is normalized for each year through min-max normalization:
- Discovered that some year, there are some anomalies for max values. 
- Particularly in 2015, the max cites/doc is 52 while in the other years, it is doubled. 
- This leads to a global inflation of normalized values for 2015 that they look much highers. There are probably other issues like this.

Thus, I decide not to normalize per year for metrics except rank:
- Rank is still normalized and inverted because the max value (rank 1 is always there).
- Rank is inverted so it is precentiles, than squared
  - The squaring is to put more distance between higher ranks, when journals are lower ranked, the difference in values do not matter so much.


In [58]:
def process_vals(vals, rank_data=0):
  '''Make sure all values are floating point numbers
  Args:
    vals (list): list of values
    def (int): 1 for rank data value that need to inverted, normalized, and
     squaared.
  Return:
    vals (list): floating point values
    vmin (float): minimum value, returned for debugging purpose
    vmax (float): maximum value, returned for debugging purpose
  '''

  # convert to float
  vals2 = []
  for v in vals:
    if type(v) == str:
      # See if three is comma that is used to indicate decimal point!!!!
      if v.find(",") == -1:
        vals2.append(float(v))
      else:
        vals2.append(float(v.replace(",", ".")))
    else:
      vals2.append(v)

  #if sum(np.isnan(vals2)) > 0:
  #  print("ERR: has NA")
  #  print(vals)
  #  isnan = np.isnan(vals2)
  #  print([vals[i] for i, x in enumerate(isnan) if x])

  vals = vals2.copy()
  vals2 = np.array(vals2)

  if rank_data == 1:
    vmax = max(vals)
    vmin = min(vals)
    # invert
    vals = [(vmax - v + vmin) for v in vals]
    # normalize
    vals = [(v - vmin) / (vmax - vmin) for v in vals]
    # square
    vals = [v*v for v in vals]
  
  vmin = min(vals)
  vmax = max(vals)

  return vals, vmin, vmax

In [59]:
def get_sjr_df(yr):
  '''Get SJR dataframe for a given year
  Args:
    yr (int): year
  Return:
    d_metric (dict): {issn: [Prank, SJR, Hidx, Cite]}
  '''

  # process SJR data for that yr
  # From 2014 and on, 2014, columns 5 (ISSN) have mixed types. 
  # Specify dtype option on import or set low_memory=False.
  sjr_file  = sjr_dir / f"scimagojr_{yr}.csv"
  sjr_df    = pd.read_csv(sjr_file, sep=";", low_memory=False)

  # get metric
  j_prank, pmin, pmax = process_vals(sjr_df['Rank'].values, 1)  
  j_sjr  , smin, smax = process_vals(sjr_df['SJR'].values)      
  j_hidx , hmin, hmax = process_vals(sjr_df['H index'].values)  
  j_cite , cmin, cmax = process_vals(sjr_df['Cites / Doc. (2years)'].values) 

  # build metric dictionary
  d_metric = {} # {issn: [Prank, SJR, Hidx, Cite]}
  issns    = sjr_df['Issn'].values
  for idx, issn in enumerate(issns):
    if issn.find(", ") != -1:
      issn = issn.split(", ")
    else:
      issn = [str(issn)]
    
    for token in issn:
      d_metric[token] = [j_prank[idx], j_sjr[idx], j_hidx[idx], j_cite[idx]]

  return d_metric, sjr_df, [pmin, pmax, smin, smax, hmin, hmax, cmin, cmax]
  

### Establish a dictionary with metrics for different years

d_d_metric: {yr: d_metric}

In [60]:
log_d_d_metric = work_dir / "log_d_d_metric.txt"

# for debuggin purpose
m_yr_min_max = {} # {yr: [pmin, pmax, smin, smax, hmin, hmax, cmin, cmax]}
with open(log_d_d_metric, "w") as f:
  f.write(f"Year\t[pmin, pmax, smin, smax, hmin, hmax, cmin, cmax]\n")
  d_d_metric = {} # {yr: d_metric}
  for yr in tqdm(yr_range):
    # Get the metric for that year
    d_metric, _, min_max = get_sjr_df(yr)
    d_d_metric[yr]       = d_metric
    m_yr_min_max[yr]     = min_max

    f.write(f"{yr}\t{min_max}\n")


  9%|▉         | 2/22 [00:00<00:03,  6.45it/s]

100%|██████████| 22/22 [00:05<00:00,  3.75it/s]


In [61]:
# Save metric dictionary as pickle
with open(work_dir / "sjr_metric_dicts.pkl", "wb") as f:
  pickle.dump(d_d_metric, f)

## ___Topical impact___

### Data processing

#### Read topic assignment and create pdjit

- Lifted from script_5_3
- Use the no dup file from 7_5
- pdjit dataframe: pmid, date, journal, issn, topic, year

In [62]:
def get_pdjit():
  print("Read topic data-frame")
  corpus_file_nodup = dir42 / 'table7_5_corpus_with_topic_assignment_nodup.tsv.gz'
  tdf = pd.read_csv(corpus_file_nodup, sep='\t', compression='gzip', index_col=[0])
  print("", tdf.shape)

  # Get pmid, date, issn, and topic
  tdf_issns = []
  not_found = []
  for journal in tdf.Journal.values:
    if journal.find("&amp;") != -1:
      journal = journal.replace("&amp;", "&")
    # anomaly with period
    # e.g., "Biology bulletin of the Academy of Sciences of the USSR. Akademiia nauk SSSR"
    #       Comptes rendus hebdomadaires des seances de l'Academie des sciences. Serie D: Sciences naturelles
    if journal.find("Comptes rendus hebdomadaires des seances de l'Academie des sciences") != -1 or \
      journal == "Development (Cambridge, England). Supplement" or \
      journal == "Nucleic acids research. Supplement (2001)":
      journal = journal.split(".")[0]
    elif journal == "Biology bulletin of the Academy of Sciences of the USSR":
      journal = "Biology bulletin of the Academy of Sciences of the USSR. Akademiia nauk SSSR"
    elif journal == "Journal of chromatography":
      journal = "Journal of chromatography. A"
    elif journal.find("Ukrains'kyi biokhimichnyi zhurnal") != -1:
      journal = "Ukrains'kyi biokhimichnyi zhurnal"
    try:
      issns = j_to_i[journal]
      tdf_issns.append(",".join(issns))
    except KeyError:
      not_found.append(journal)
      tdf_issns.append("")

  print("Len not found:", len(not_found))
  print(sorted(not_found))
  print("Len tdf.Journal.values, tdf_issns")
  print(len(tdf.Journal.values), len(tdf_issns))

  # Create dataframe with pmid, date, journal, topic
  pdjit = tdf[['PMID', 'Date', 'Journal', 'Topic']]

  # Insert ISSN
  pdjit.insert(3, 'ISSN', tdf_issns)

  # Add year to pdjit
  year = [int(date.split('-')[0]) for date in pdjit['Date'].values]
  pdjit['Year'] = year

  return pdjit

In [63]:

# Check if this file is already generated
pdjit_file = work_dir / 'table_pdjity.tsv'
if pdjit_file.is_file():
  print("pdjit file exists")
else:
  pdjit = get_pdjit()
  pdjit.to_csv(pdjit_file, sep='\t', index=False)

# Regardless if the file was generated or not, read it.
pdjit = pd.read_csv(pdjit_file, sep='\t')
pdjit.head(2)

Read topic data-frame
 (421307, 12)
Len not found: 163
['Acta biochimica et biophysica; Academiae Scientiarum Hungaricae', 'Acta biochimica et biophysica; Academiae Scientiarum Hungaricae', 'Acta physiologiae plantarum', 'Acta physiologiae plantarum', 'Acta physiologiae plantarum', 'Acta physiologiae plantarum', 'Acta physiologiae plantarum', 'Acta physiologiae plantarum', 'Acta physiologiae plantarum', 'Acta physiologiae plantarum', 'Acta physiologiae plantarum', 'Agricultural systems', 'Agricultural systems', 'Ai zheng = Aizheng = Chinese journal of cancer', 'Ai zheng = Aizheng = Chinese journal of cancer', 'Applied mathematical modelling', 'Bulletin de la Societe de pathologie exotique et de ses filiales', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH protocols', 'CSH prot

/tmp/ipykernel_1165/4002324010.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdjit['Year'] = year


,PMID,Date,Journal,ISSN,Topic,Year
0,61,1975-12-11,Biochimica et biophysica acta,"00063002,18782434",52,1975
1,67,1975-11-20,Biochimica et biophysica acta,"00063002,18782434",48,1975


#### Get topc indices and names

In [64]:
# topic assignments
toc_array = pdjit['Topic'].values

# topic indices
tocs = np.unique(toc_array)

# exclude topic=-1
tocs_90 = tocs[1:]

# number of topic=-1
n_rec_toc_unassigned = sum((toc_array==-1).astype(int))

# number of docs with topic assignment. Originally was thinking about minus
# unassigned, but realize that the totol for taxa would be the number of total
# docs, so it does not make sense to remove unassigned.
n_rec_total  = len(toc_array)

print("number of topic=-1:", n_rec_toc_unassigned)
print("number of docs with topic assignment:", n_rec_total)

number of topic=-1: 49192
number of docs with topic assignment: 421307


In [110]:
toc_names = pd.read_csv(file_topic_name, sep='\t')
toc_names.head(2)

,Topic,Mod_name
0,22,enzyme | fatty acids | lipid | synthesis
1,18,protein | dna | rna | synthesis | mrna


In [112]:
toc_names[toc_names['Topic'] == 0]['Mod_name'].values[0]

'allergen | pollen | ige | allergenic'

#### Get topic-pub number dataframe

Only do this for the yr_range (1999~2020)
- Topic, PubNum, Mod_name

In [78]:
pdjit_sub = pdjit.loc[(pdjit['Year']>=1999) & (pdjit['Year']<=2022)]
pdjit.shape, pdjit_sub.shape

((421307, 6), (351157, 6))

In [114]:
toc_pub_num_file = work_dir / 'toc_pub_num.csv'

list_pub_num = []
list_toc_nms = []
# exclude topic = -1
for toc in tocs[1:]:
  df = pdjit.loc[(pdjit['Topic']==toc) & \
                (pdjit['Year']>=1999) & \
                (pdjit['Year']<=2022)]
  toc_name = toc_names[toc_names['Topic'] == toc]['Mod_name'].values[0]
  list_pub_num.append(df.shape[0])
  list_toc_nms.append(toc_name)

df_pub_num = pd.DataFrame({'Topic': tocs[1:], 
                           'PubNum': list_pub_num,
                           'TopicName': list_toc_nms})
df_pub_num.to_csv(toc_pub_num_file)

In [115]:
df_pub_num.head(2)

,Topic,PubNum,TopicName
0,0,585,allergen | pollen | ige | allergenic
1,1,1615,medium | callus | regeneration | culture | som...


### Get topical impact for each topic, each year

#### Get metric lists for each topic

4 metrics in order:
- Percentile rank
- SJR
- H-index
- Cite/doc

In [65]:
# Taken from 8.2
def get_m_list(pmid, journal, issn, d_metric, not_found):
  
  # first check if issn is np.nan
  if type(issn) == float:
    if not np.isnan(issn):
      print("ERR: float but not nan", issn)
    issn = []
  else:
    issn = issn.split(",")

  # make sure issn, if exist, is in d_metric, then append to a metric list
  m_list = []
  for issn_token in issn:
    if issn_token in d_metric:
      metrics = d_metric[issn_token]
      m_list.append(metrics)

  # check if this journal is found in d_meric
  m_list2 = [] # initialize m_list2
  if m_list == []:
    # not found, add to a dictionary for logging purpose
    if journal not in not_found:
      not_found[journal] = [issn, [pmid]]
    else:
      not_found[journal][1].append(pmid)
  else:
    # get average if multiple issns
    for idx in range(0,4):
      m_sum = 0
      for ms in m_list:
        m_sum += ms[idx]
      m_avg = m_sum / len(m_list)
      m_list2.append(m_avg)

  return m_list, m_list2

In [66]:
def get_m_lst_lst(toc, yr):
  '''Get a list of metric lists for a given topic and year
  Args:
    toc (int): topic index
    yr (int): year
  Return:
    m_lst_lst (list): [m_lst], where m_lst = [pmid, journal, issn, [Prank, SJR,
      Hidx, Cite]]
    not_found (dict): {journal: [issn, [pmids]]}
  '''
  
  # Get sub-dataframe
  # If toc is empty, get all topics for that year
  if toc == "":
    df = pdjit.loc[pdjit['Year']==yr]
  else:
    df = pdjit.loc[(pdjit['Topic']==toc) & (pdjit['Year']==yr)]

  # pmids, journals, and issns
  issns     = df['ISSN'].values
  journals  = df['Journal'].values
  pmids     = df['PMID'].values

  d_metric  = d_d_metric[yr] # {issn: [Prank, SJR, Hidx, Cite]}
  not_found = {}             # {journal: [issn, [pmids]}
  
  m_lst_lst = []  # [m_lst]
  for idx, issn in enumerate(issns):
    journal = journals[idx]
    pmid    = pmids[idx]

    m_list, m_list2 = get_m_list(pmid, journal, issn, d_metric, not_found)

    if m_list != []:
      # need m_list2, but add more info for debugging
      m_lst_lst.append([pmid,journal,issn,m_list2, m_list])

  return m_lst_lst, not_found

#### Calculate average

Determine average impact
- For pubs in topic T in year Y
  - Go through journal of each pub
  - Get the metric for that journal
  - Add the metrics
  - Divid the metric total by number of pubs in topic T in year Y

In [67]:
tmp_m_lst_lst, _ = get_m_lst_lst(0, 1999)
len(tmp_m_lst_lst), tmp_m_lst_lst[0]

(34,
 [9874211,
  'European journal of biochemistry',
  '00142956,14321033',
  [0.9383876871528496, 2.227, 222.0, 3.26],
  [[0.9383876871528496, 2.227, 222, 3.26]]])

In [68]:
log_toc_yr_issn_not_found = work_dir / "log_toc_yr_issn_not_found.txt"

with open(log_toc_yr_issn_not_found, "w") as f:
  t_y_avg = {} # {topic: {year: [Prank, SJR, Hidx, Cite]}}

  # For each topic
  zero_pub = []
  for toc in tqdm(tocs):
    # output to log file
    f.write(f"-----\ntopic={toc}\n")
    t_y_avg[toc] = {}

    # For each year
    for yr in yr_range:
      # [[prank, sjr, hidx, cite]] for all records in a given topic-year
      m_lst_lst, not_found = get_m_lst_lst(toc, yr) 
      #print(len(m_lst_lst))
      
      # compile metrics into a 2d array
      m_2d = []
      for m_list in m_lst_lst:
        m_2d.append(m_list[3])
      m_2d  = np.array(m_2d)

      # determine n_pub for each metric after removing NA
      n_pub = np.subtract([m_2d.shape[0]]*4, sum(np.isnan(m_2d)))
      
      # For a few cases without publication, set to NaN
      if 0 in n_pub:
        t_y_avg[toc][yr] = [np.nan]*4
        zero_pub.append([toc, yr])
      else:
        # calculate average and store in dict
        # Issue: RuntimeWarning: invalid value encountered in true_divide
        # https://www.geeksforgeeks.org/how-to-fix-invalid-value-encountered-in-true_divide/
        m_sum = np.nansum(m_2d, axis=0)
        m_avg = np.divide(m_sum, n_pub)
        t_y_avg[toc][yr] = m_avg
      
      # output to log file
      f.write(f" {yr}:\n")
      for journal in not_found:
        f.write(f"  {journal}:\n" + \
                f"    ISSN:{not_found[journal][0]}\n" + \
                f"    PMID:{not_found[journal][1]}\n")

100%|██████████| 91/91 [00:06<00:00, 13.14it/s]


In [69]:
zero_pub

[[5, 1999],
 [5, 2000],
 [5, 2002],
 [5, 2003],
 [5, 2004],
 [5, 2006],
 [5, 2007],
 [5, 2008],
 [5, 2009],
 [5, 2011],
 [5, 2012],
 [5, 2015],
 [8, 2001],
 [8, 2004],
 [8, 2007],
 [8, 2008]]

#### Generate output

Build dataframes
- A datafame for each of metric: prank, sjr, hidx, cite

Use dataframe to generate an xlsx with 4 sheets:
- One sheet for each of Prank, SJR, Hidx, Cite
- Rows are topics, columns are years
- Decide not to do min-max norm for each topic as this. The point is between topic comparison, similar to the country one.
- Sorted according to toc_pub_num

In [70]:
t_y_avg[0][1999]

array([  0.79302746,   1.40879412, 229.02941176,   2.77529412])

In [90]:
df_pub_num_sort = df_pub_num.sort_values(by='PubNum', ascending=False)
toc_sort        = df_pub_num_sort['Topic'].tolist()

df_pub_num_sort.head(3)

,Topic,PubNum
61,61,13466
79,79,9951
42,42,9632


In [91]:
toc_sort[:3]

[61, 79, 42]

In [92]:
toc_excel_file    = work_dir / 'impact_topics.xlsx'
toc_excel_writer  = pd.ExcelWriter(toc_excel_file, engine='xlsxwriter')

metric_names = ['prank', 'sjr', 'hidx', 'cite']
for metric_idx in range(4):
  metric_nm      = metric_names[metric_idx]
  metric_2d      = [] # a 2D list: toc, then year
  #metric_2d_norm = [] # a 2D list: toc, then year, normalized for each toc

  # Get metrics for a topic based on a sorted list
  for toc in toc_sort:
    metric_toc = []
    for yr in yr_range:
      metric_toc.append(t_y_avg[toc][yr][metric_idx])
    metric_2d.append(metric_toc)

    # do min-max normalization
    #m_min = min(metric_toc)
    #m_max = max(metric_toc)
    #metric_toc_norm = [(m-m_min)/(m_max-m_min) for m in metric_toc]
    #metric_2d_norm.append(metric_toc_norm)
  
  df_metric  = pd.DataFrame(metric_2d, index=toc_sort, columns=yr_range)
  #df_metric2 = pd.DataFrame(metric_2d_norm, index=tocs, columns=yr_range)
  print(metric_nm, df_metric.shape)
  
  df_metric.to_excel(toc_excel_writer, sheet_name=metric_nm)
  #df_metric2.to_excel(toc_excel_writer, sheet_name=metric_nm+"_norm")

toc_excel_writer.close()
  

prank (90, 22)
sjr (90, 22)
hidx (90, 22)
cite (90, 22)


## ___Overall impact___

Follow 8.2 to get overall impact each year for normalizing topics

### Get yearly average impact

In [72]:
tmp_m_lst_lst, _ = get_m_lst_lst("", 1999)
len(tmp_m_lst_lst), tmp_m_lst_lst[0]

(4885,
 [9852189,
  'Environmental management',
  '0364152X,14321009',
  [0.6865889944216469, 0.723, 135.0, 1.04],
  [[0.6865889944216469, 0.723, 135, 1.04],
   [0.6865889944216469, 0.723, 135, 1.04]]])

In [73]:
# lifted from 8.2
y_avg = {} # {year: [Prank, SJR, Hidx, Cite]}

# For each year
for yr in tqdm(yr_range):
  # [[prank, sjr, hidx, cite]] for all records in a given topic-year
  m_lst_lst, _ = get_m_lst_lst("", yr) 
  #print(yr, len(m_lst_lst))
  
  # compile metrics into a 2d array
  m_2d = []
  for m_list in m_lst_lst:
    # m_list2 is in index=3, unlike the country one
    m_2d.append(m_list[3])

  m_2d  = np.array(m_2d)

  # determine n_pub for each metric after removing NA
  n_pub = np.subtract([m_2d.shape[0]]*4, sum(np.isnan(m_2d)))
  
  # For a few cases without publication, set to NaN
  if 0 in n_pub:
    y_avg[yr] = [np.nan]*4
  else:
    # calculate average and store in dict
    # Issue: RuntimeWarning: invalid value encountered in true_divide
    # https://www.geeksforgeeks.org/how-to-fix-invalid-value-encountered-in-true_divide/
    m_sum = np.nansum(m_2d, axis=0)
    m_avg = np.divide(m_sum, n_pub)
    y_avg[yr] = m_avg
    

100%|██████████| 22/22 [00:02<00:00,  7.57it/s]


In [74]:
y_avg

{1999: array([  0.86648555,   3.16141952, 259.0321392 ,   3.72579734]),
 2000: array([  0.81659461,   2.54369294, 238.68190241,   3.37575973]),
 2001: array([  0.7946001 ,   2.26468397, 232.47058824,   3.24850718]),
 2002: array([  0.7918344 ,   2.20795582, 227.79034865,   3.35888017]),
 2003: array([  0.78830859,   2.16984901, 217.3126981 ,   3.19734746]),
 2004: array([  0.79378397,   2.16655436, 216.31421947,   3.4100814 ]),
 2005: array([  0.77013863,   2.03226346, 205.51933505,   3.44098142]),
 2006: array([  0.79333285,   2.14554766, 208.95209838,   3.54732456]),
 2007: array([  0.78555407,   2.19558925, 204.38862406,   3.58484285]),
 2008: array([  0.78727613,   2.07493528, 195.74775412,   3.36942597]),
 2009: array([  0.79193669,   2.19646635, 198.140281  ,   3.60986059]),
 2010: array([  0.79179347,   2.10483888, 195.34242361,   3.66531886]),
 2011: array([  0.796995  ,   2.06188168, 195.81327362,   3.79105803]),
 2012: array([  0.80233504,   2.04369828, 204.01059334,   3.9383

### Generate output

In [75]:
excel_file2    = work_dir / 'impact_all_toc_per_year.xlsx'
excel_writer2  = pd.ExcelWriter(excel_file2, engine='xlsxwriter')

metric_names = ['prank', 'sjr', 'hidx', 'cite']
metric_2d    = [] # a 2D list: year, then metrics
for yr in yr_range:
  metric_2d.append(y_avg[yr])

df_metric = pd.DataFrame(metric_2d, index=yr_range, columns=metric_names)
df_metric.to_excel(excel_writer2, sheet_name="yearly overall")

excel_writer2.close()

## ___Testing___

### Check anomalies

In the spreadsheet, there are some strange patterns:
- cite_norm: 2015 is abnormally high compared to neighboring years.
- sjr_norm: 2019 is almost all 0, 2015 is abnormally high (though not as bad as cite_norm)

Ok, look at log_d_d_metric.txt, the issue is with c_max (cites/doc max value) that is half of others.
- year [pmax, pmin, smin, smax, hmax, hmin, cmin, cmax]
- 2014	[1, 33044, 0.1, 47.751, 0, 1331, 0.0, 117.0]
- 2015	[1, 33787, 0.1, 35.501, 0, 1331, 0.0, 52.14]
- 2016	[1, 34401, 0.1, 43.002, 0, 1331, 0.0, 101.2]

#### Check avg and median cites/doc

Looks fine

In [39]:
def get_stat(year, field):
  sjr_file = sjr_dir / f"scimagojr_{year}.csv"
  sjr_df   = pd.read_csv(sjr_file, sep=";", low_memory=False)
  vals     = sjr_df[field].values

  #print(sjr_df.dtypes)

  # convert to float
  vals2 = []
  for v in vals:
    if type(v) == str:
      # See if three is comma that is used to indicate decimal point!!!!
      if v.find(",") == -1:
        vals2.append(float(v))
      else:
        vals2.append(float(v.replace(",", ".")))
    else:
      vals2.append(v)

  sjr_nan  = np.sum(np.isnan(vals2))
  sjr_med  = np.nanmedian(vals2)
  sjr_avg  = np.nanmean(vals2)

  print(f"{year}-{field}: med={sjr_med}, avg={sjr_avg}, n_nan={sjr_nan}")

In [40]:
get_stat(2014, "Cites / Doc. (2years)")
get_stat(2015, "Cites / Doc. (2years)")
get_stat(2016, "Cites / Doc. (2years)")

2014-Cites / Doc. (2years): med=0.45, avg=1.0064662268490496, n_nan=0
2015-Cites / Doc. (2years): med=0.47, avg=1.0252990795276289, n_nan=0
2016-Cites / Doc. (2years): med=0.51, avg=1.0648251504316735, n_nan=0


#### Check pubs in topic 0 for cites/doc

Ok, for toc=0, there is a large difference in mean and median between 2014, 2015, and 2016.
- These are based on normalized values, something wrong with normalization?
- Then why just 2015?

In [41]:
toc0_2014, nf_2014 = get_m_lst_lst(0, 2014)
toc0_2015, nf_2015 = get_m_lst_lst(0, 2015)
toc0_2016, nf_2016 = get_m_lst_lst(0, 2016)

In [42]:
toc0_2014[:2]

[[24383968,
  'Asian Pacific journal of allergy and immunology',
  ['0125877X'],
  [0.5194481823679986, 0.483, 39.0, 1.21],
  [[0.5194481823679986, 0.483, 39, 1.21]]],
 [24468678,
  'International immunopharmacology',
  ['15675769', '18781705'],
  [0.7741422240396723, 1.061, 126.0, 2.83],
  [[0.7741422240396723, 1.061, 126, 2.83],
   [0.7741422240396723, 1.061, 126, 2.83]]]]

In [43]:
# So journal not found is similar
len(nf_2014), len(nf_2015), len(nf_2016)

(1, 1, 0)

In [44]:
def get_cites_per_doc_list(m_lst_lst):
  cpd_list = [rec[3][3] for rec in m_lst_lst]
  cpd_nan  = np.sum(np.isnan(cpd_list))
  cpd_avg  = np.nanmean(cpd_list)
  cpd_med  = np.nanmedian(cpd_list)
  print(f"len:{len(cpd_list)},nan={cpd_nan},avg={cpd_avg},med={cpd_med}")


In [45]:
get_cites_per_doc_list(toc0_2014)
get_cites_per_doc_list(toc0_2015)
get_cites_per_doc_list(toc0_2016)

len:28,nan=0,avg=3.236428571428571,med=3.0549999999999997
len:31,nan=0,avg=3.766451612903225,med=3.51
len:21,nan=0,avg=2.8785714285714286,med=2.86


#### Check same journal across years

Problem: very few journals are the same across years. But for those found in at least two years
- There are generally 2x higher values in 2015.
- E.g., PloS one [0.03170940170940171, 0.06731875719217491, nan]
  - Look at SJR record, Cites / Doc. (2years)
    - 2014: 3.71
    - 2015: 3.51
  - These are fine. So something is not right about on how these values are normalized or populated.

In [46]:
def build_j_cpd(j_cpd, m_lst_lst, yr):
  idx = 0
  if yr == 2015:
    idx = 1
  elif yr == 2016:
    idx = 2
  
  for m_lst in m_lst_lst:
    journal = m_lst[1]
    metric  = m_lst[3][3]
    if journal in j_cpd:
      j_cpd[journal][idx] = metric
    else:
      j_cpd[journal] = [np.nan, np.nan, np.nan]
      j_cpd[journal][idx] = metric

  return j_cpd

In [47]:
j_cpd = {} # {journal:[cpd2014, cpd2015, cpd2016]}
j_cpd = build_j_cpd(j_cpd, toc0_2014, 2014)
j_cpd = build_j_cpd(j_cpd, toc0_2015, 2015)
j_cpd = build_j_cpd(j_cpd, toc0_2016, 2016)

In [48]:
# Look at Plos one as an example
for j in j_cpd:
  if sum(np.isnan(j_cpd[j])) < 2:
    print(j, "\t", j_cpd[j])

Molecular nutrition &amp; food research 	 [4.91, nan, 4.54]
Clinical and experimental allergy : journal of the British Society for Allergy and Clinical Immunology 	 [4.78, nan, 4.69]
Allergy 	 [6.67, 6.77, nan]
Journal of agricultural and food chemistry 	 [3.28, 3.19, 3.48]
BMC plant biology 	 [4.4, nan, 4.31]
The Journal of allergy and clinical immunology 	 [7.18, 7.37, nan]
Journal of investigational allergology &amp; clinical immunology 	 [1.45, 1.27, nan]
PloS one 	 [3.71, 3.51, nan]
Scientific reports 	 [nan, 5.78, 4.65]
Biomolecular NMR assignments 	 [nan, 0.67, 0.4]


#### Check just Plos one

The original cites/doc values looks normal in 2015, not off from other years
- Point to issue with normalization again.

In [49]:
toc0_2014[0][1]

'Asian Pacific journal of allergy and immunology'

In [50]:
for rec in toc0_2014:
  if rec[1] == "PloS one":
    print(rec[3][3], rec[4][0][3])

3.71 3.71


In [51]:
for rec in toc0_2015:
  if rec[1] == "PloS one":
    print(rec[3][3], rec[4][0][3])

3.51 3.51
3.51 3.51
3.51 3.51
3.51 3.51
3.51 3.51


#### Check SJR data

In [52]:
found_issn = 0
not_found = []
for issns in df.Issn:
  issns = issns.strip().split(", ")
  found = 0
  for issn in issns:
    if issn in i_to_j:
      found_issn += 1
      found = 1
      break
  
  if found:
    found = 0
  else:
    not_found.append(issn)

print(f"Total ISSNs: {len(df)}, in Pubmed: {found_issn}")

Total ISSNs: 18024, in Pubmed: 10588


In [53]:
not_found[:10]

['09642536',
 '15734463',
 '10950761',
 '03044157',
 '00796379',
 '-',
 '08940347',
 '15740048',
 '13864181',
 '00653055']

#### Check SJR data calculation

Example:
- [pmid, journal, [issn], [Prank, SJR, H-idx, Cite/doc]]
- [9872323, 'Oncogene', ['09509232', '14765594'], [0.9908989489753978, 0.09022571303899402, 0.270473328324568, 0.13452256033578175]]

Nothing going on here.



In [54]:
sjr_file  = sjr_dir / f"scimagojr_1999.csv"
sjr_df    = pd.read_csv(sjr_file, sep=";", low_memory=False)
sjr_df[sjr_df["Title"] == "Oncogene"]

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (1999),Total Docs. (3years),...,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Areas
155,156,12523,Oncogene,journal,"14765594, 09509232","4,649",Q1,360,924,2088,...,13818,2087,"6,41","49,54",United Kingdom,Western Europe,Nature Publishing Group,1987-2022,Cancer Research (Q1); Genetics (Q1); Molecular...,"Biochemistry, Genetics and Molecular Biology"


In [59]:
def get_val(journal, yr):
  m_vals = []
  # metric type
  m_types = ["Rank", "SJR", "H index", "Cites / Doc. (2years)"]
  for m_type in m_types:
    val = sjr_df[sjr_df["Title"] == journal][m_type].values[0]
    if type(val) == str and val.find(",") != -1:
      val = val.replace(",", ".")
    m_vals.append(float(val))

  print(m_vals)

  min_max = m_yr_min_max[yr]
  print(process_vals([min_max[0], min_max[1], m_vals[0]], rank_data=1))
  print(process_vals([min_max[2], min_max[3], m_vals[1]]))
  print(process_vals([min_max[4], min_max[5], m_vals[2]]))
  print(process_vals([min_max[6], min_max[7], m_vals[3]]))

In [60]:
get_val("Oncogene", 1999)

[156.0, 4.649, 360.0, 6.41]
([1.0, 0.9872205785667325, 0.0], 0.0, 1.0)
([0.1, 50.518, 4.649], 0.1, 50.518)
([0, 1331, 360.0], 0, 1331)
([0.0, 47.65, 6.41], 0.0, 47.65)


In [61]:
sjr_df[sjr_df["Title"] == "Plant Cell"]

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (1999),Total Docs. (3years),...,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Areas
52,53,16594,Plant Cell,journal,"1532298X, 10404651","8,834",Q1,380,196,535,...,5620,522,"9,94","54,43",United States,Northern America,Oxford University Press,1989-2022,Cell Biology (Q1); Plant Science (Q1),Agricultural and Biological Sciences; Biochemi...


In [62]:
get_val("Plant Cell", 1999)

[53.0, 8.834, 380.0, 9.94]
([1.0, 0.9626201495194019, 0.0], 0.0, 1.0)
([0.1, 50.518, 8.834], 0.1, 50.518)
([0, 1331, 380.0], 0, 1331)
([0.0, 47.65, 9.94], 0.0, 47.65)


### Get topic=0, year=2001

In [65]:
len(tocs), len(yr_range), pdjit.head(2)

(91,
 22,
    PMID        Date                        Journal               ISSN  Topic  \
 0    61  1975-12-11  Biochimica et biophysica acta  00063002,18782434     52   
 1    67  1975-11-20  Biochimica et biophysica acta  00063002,18782434     48   
 
    Year  
 0  1975  
 1  1975  )

In [66]:
pdjit.loc[(pdjit['Topic']==0) & (pdjit['Year']==2001)]

,PMID,Date,Journal,ISSN,Topic,Year
48287,11139585,2001-01-15,The Journal of biological chemistry,"00219258,1083351X",0,2001
49696,11251404,2001-03-17,Allergy,"01054538,13989995",0,2001
49697,11251633,2001-03-17,Clinical and experimental allergy : journal of...,"09547894,13652222",0,2001
49698,11251634,2001-03-17,Clinical and experimental allergy : journal of...,"09547894,13652222",0,2001
49936,11270469,2001-03-29,Asian Pacific journal of allergy and immunology,0125877X,0,2001
50467,11306929,2001-04-18,International archives of allergy and immunology,"10182438,14230097",0,2001
50973,11344353,2001-05-10,The Journal of allergy and clinical immunology,"00916749,10976825",0,2001
51031,11350307,2001-05-15,Allergy,"01054538,13989995",0,2001
51907,11419719,2001-06-23,"Journal of chromatography. B, Biomedical scien...","13872273,18785603",0,2001
51908,11419723,2001-06-23,"Journal of chromatography. B, Biomedical scien...","13872273,18785603",0,2001


### Deal with nan

In [67]:
arr = np.array([5,4,2,2,4,np.nan,np.nan,6])
np.isnan(arr)

array([False, False, False, False, False,  True,  True, False])

In [68]:
arr[np.isnan(arr)]

array([nan, nan])

### Deprecated

In [ ]:
# 8/7/23, replaced by two functions get_m_lst_lst and get_m_list
def get_m_lst_lst(toc, yr):
  '''Get a list of metric lists for a given topic and year
  Args:
    toc (int): topic index
    yr (int): year
  Return:
    m_lst_lst (list): [m_lst], where m_lst = [pmid, journal, issn, [Prank, SJR,
      Hidx, Cite]]
    not_found (dict): {journal: [issn, [pmids]]}
  '''
  
  # Get sub-dataframe
  # If toc is empty, get all topics for that year
  if toc == "":
    df = pdjit.loc[pdjit['Year']==yr]
  else:
    df  = pdjit.loc[(pdjit['Topic']==toc) & (pdjit['Year']==yr)]

  # pmids, journals, and issns
  issns     = df['ISSN'].values
  journals  = df['Journal'].values
  pmids     = df['PMID'].values

  d_metric  = d_d_metric[yr] # {issn: [Prank, SJR, Hidx, Cite]}
  not_found = {}             # {journal: [issn, [pmids]}
  
  m_lst_lst = []  # [m_lst]
  for idx, issn in enumerate(issns):
    journal = journals[idx]
    pmid    = pmids[idx]

    # first check if issn is np.nan
    if type(issn) == float:
      if not np.isnan(issn):
        printO("ERR: float but not nan", issn)
      issn = []
    else:
      issn = issn.split(",")

    # make sure issn, if exist, is in d_metric, then append to a metric list
    m_list = []  
    for issn_token in issn:
      if issn_token in d_metric:
        metrics = d_metric[issn_token]
        m_list.append(metrics)

    # check if this journal is found in d_meric
    if m_list == []:
      # not found, add to a dictionary for logging purpose
      if journal not in not_found:
        not_found[journal] = [issn, [pmid]]
      else:
        not_found[journal][1].append(pmid)
    else:
      # get average if multiple issns
      m_list2 = []
      for idx in range(0,4):
        m_sum = 0
        for ms in m_list:
          m_sum += ms[idx]
        m_avg = m_sum / len(m_list)
        m_list2.append(m_avg)

      # need m_list2, but add more info for debugging
      m_lst_lst.append([pmid,journal,issn,m_list2, m_list])

  return m_lst_lst, not_found